In [22]:
import os
import kfp

In [23]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_training_op = kfp.components.load_component_from_file(
    components_url + "training/component.yaml"
)
dkube_serving_op = kfp.components.load_component_from_file(
    components_url + "serving/component.yaml"
)

token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(
    host=os.getenv("KF_PIPELINES_ENDPOINT"),
    existing_token=token,
    namespace=os.getenv("USERNAME"),
)
run_id = 0

In [24]:
@kfp.dsl.pipeline(
    name='fireweather',
    description='fireweather'
)

def fireweather_pipeline():

    # -------------------------------
    # TRAINING STEP
    # -------------------------------
    train = dkube_training_op(
        auth_token=token,
        container='{"image":"ocdr/d3-datascience-sklearn:v0.23.2-1"}',
        framework="sklearn",
        version="0.23.2",
        program="project2",                  # Your DKube code repo
        run_script="sleep 10m; python training.py",        # Your training script
        datasets='["dataset2"]',          # NAME OF DATASET IN DKUBE
        input_dataset_mounts='["/mnt/ou/"]',    # Will mount dataset at /data
        outputs='["fireweather"]',             # Output model artifact
        output_mounts='["/mnt/model"]',          # Save model to /model
        envs='[]'
    )

    # -------------------------------
    # SERVING STEP
    # -------------------------------
    serving = dkube_serving_op(
        auth_token=token,
        model=train.outputs["artifact"],     # Model produced by training
        device="cpu",
        serving_image='{"image":"ocdr/sklearnserver:0.23.2"}',
#         transformer_image="",                 # No transformer needed
#         transformer_project="",               # empty
#         transformer_code=""                   # empty
    ).after(train)


In [21]:
run_name = f"[project2] Run{run_id}"
experiment = f"[project2] experiment"

client.create_run_from_pipeline_func(
    fireweather_pipeline,        # <-- your pipeline function
    run_name=run_name,
    experiment_name=experiment,
    arguments={"project_id":project_id}
)

run_id += 1


NameError: name 'project_id' is not defined

In [25]:
client.create_run_from_pipeline_func(fireweather_pipeline, arguments={})

RunPipelineResult(run_id=cc49ba73-45c4-4c64-86e4-77051e46820c)

In [15]:
pwd

'/home/ocdkube1/workspace/project2'